In [3]:
import fitz  #PyMuPDF: Para leer archivos PDF
import re  #Expresiones regulares: Para buscar fechas
import pytesseract  #OCR: Para extraer texto de imágenes
import os  #Para trabajar con archivos y carpetas
import pandas as pd  #Para manejar los datos en tablas
from PIL import Image  #Para abrir y manipular imágenes
from datetime import datetime  #Para trabajar con fechas

#Ruta de la carpeta que contiene los recibos
folder_path = r"C:\Users\Federico Labate\Desktop\Pagos de alquiler\Recibos"

#Función para extraer texto de un archivo PDF
def extraer_texto_pdf(pdf_path):
    #Abro el archivo PDF
    doc = fitz.open(pdf_path)
    texto = ""
    #Recorro todas las páginas del PDF y extraigo el texto
    for page in doc:
        texto += page.get_text("text")
    return texto

#Función para extraer texto de una imagen usando OCR
def extraer_texto_imagen(image_path):
    #Uso pytesseract para extraer texto de la imagen
    return pytesseract.image_to_string(Image.open(image_path), lang="spa")

#Función para buscar fechas en un texto usando expresiones regulares
def extraer_fecha(texto):
    #Defino patrones de fechas buscadas
    patrones_fecha = [
        r"\b(\d{2}-\d{2}-\d{4})\b",  #DD-MM-YYYY
        r"\b(\d{2}/\d{2}/\d{2})\b",   #DD/MM/YY
        r"\b(\d{2}) (enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre) (\d{4})\b",  #DD mes YYYY
        r"\b(\d{2}\.\d{2}\.\d{4})\b"  #DD.MM.YYYY
    ]
    
    #Bucle para encontrar una fecha en el texto
    for patron in patrones_fecha:
        match = re.search(patron, texto, re.IGNORECASE)
        if match:
            fecha_str = match.group(0)  #Si encuentro una fecha, la guardo
            try:
                #Dependiendo del formato de la fecha, la convierto a un objeto datetime
                if "-" in fecha_str:
                    return datetime.strptime(fecha_str, "%d-%m-%Y")
                elif "/" in fecha_str and len(fecha_str.split("/")[-1]) == 2:
                    return datetime.strptime(fecha_str, "%d/%m/%y")
                elif "." in fecha_str:
                    return datetime.strptime(fecha_str, "%d.%m.%Y")
                else:
                    dia, mes, año = match.groups()  #Extraigo el día, mes y año
                    meses = {
                        "enero": "01", "febrero": "02", "marzo": "03", "abril": "04", "mayo": "05", "junio": "06",
                        "julio": "07", "agosto": "08", "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
                    }
                    #Si el mes está en formato texto (enero, febrero, etc.), lo convierto a número
                    return datetime.strptime(f"{dia}-{meses[mes.lower()]}-{año}", "%d-%m-%Y")
            except ValueError:
                pass
    
    return None  #Si no encuentro ninguna fecha válida, devuelvo None

#Función para procesar un archivo (puede ser PDF o imagen) y extraer la fecha
def procesar_archivo(file_path):
    #Si el archivo es PDF, llamo a la función para extraer texto del PDF
    if file_path.lower().endswith(".pdf"):
        texto = extraer_texto_pdf(file_path)
    else:  #Si el archivo es imagen, llamo a la función para extraer texto de la imagen
        texto = extraer_texto_imagen(file_path)
    
    #Extraigo la fecha del texto obtenido
    return extraer_fecha(texto)

#Función para procesar todos los archivos de una carpeta
def procesar_carpeta(folder_path):
    resultados = []  #Creo una lista vacía para almacenar los resultados
    meses_encontrados = set()  #Uso un set para evitar duplicados en los meses encontrados
    
    #Recorro todos los archivos en la carpeta
    for root, _, files in os.walk(folder_path):
        for file in files:
            #Si el archivo es PDF o imagen (JPEG o JPG)
            if file.lower().endswith((".pdf", ".jpg", ".jpeg")):
                file_path = os.path.join(root, file)  #Obtengo la ruta completa del archivo
                fecha = procesar_archivo(file_path)  #Extraigo la fecha del archivo
                if fecha:  #Si se encontró una fecha, la fecha de pago esperada será el día 5 del mes siguiente
                    if fecha.month == 12: #Si el mes es diciembre, la fecha de pago esperada será en enero del año siguiente
                        fecha_pago_esperada = fecha.replace(year=fecha.year + 1, month=1, day=5)
                    else:
                        fecha_pago_esperada = fecha.replace(month=fecha.month + 1, day=5)
                    
                    #Calculo la diferencia de días entre la fecha de pago esperada y la fecha actual
                    delta_pago = (fecha_pago_esperada - fecha).days
                    
                    #Guardo los datos de interés en un diccionario
                    resultados.append({
                        "Año_mes": f"{fecha.year}_{fecha.month:02d}",  #Guardo en formato YYYY_MM para facilitar el orden
                        "Fecha de pago esperada": fecha_pago_esperada.strftime('%Y-%m-%d'),
                        "Fecha de Pago": fecha.strftime('%Y-%m-%d'),
                        "Delta de pago": delta_pago
                    })
                    #Añado el mes encontrado al set
                    meses_encontrados.add(f"{fecha.year}_{fecha.month:02d}")

    return resultados, meses_encontrados  #Devuelvo los resultados y los meses encontrados

#Llamo a la función con los resultados procesados de los meses encontrados
resultados, meses_encontrados = procesar_carpeta(folder_path)

#Convierto los resultados a un df
df = pd.DataFrame(resultados)

#Ordeno el df por la columna "Año_mes"
df['Año_mes'] = df['Año_mes'].astype(str)  #Aseguro que 'Año_mes' sea un string
df = df.sort_values(by="Año_mes")  #Ordeno los resultados por "Año_mes" de menor a mayor

#Defino los meses que faltan y que debo agregar manualmente (Sin fecha en los recibos)
meses_faltantes_manual = {
    "2021_01": {"fecha_pago_esperada": "2021-02-05", "fecha_pago": "2021-01-29"},
    "2022_03": {"fecha_pago_esperada": "2022-03-05", "fecha_pago": "2022-02-28"},
    "2022_06": {"fecha_pago_esperada": "2022-06-05", "fecha_pago": "2022-05-20"},
    "2022_12": {"fecha_pago_esperada": "2022-12-05", "fecha_pago": "2022-11-29"},
    "2023_11": {"fecha_pago_esperada": "2023-11-05", "fecha_pago": "2023-10-30"}
}

#Agrego manualmente los meses faltantes
for mes, fechas in meses_faltantes_manual.items():
    if mes not in meses_encontrados:
        fecha_pago_esperada = datetime.strptime(fechas["fecha_pago_esperada"], "%Y-%m-%d")
        fecha_pago = datetime.strptime(fechas["fecha_pago"], "%Y-%m-%d")
        delta_pago = (fecha_pago_esperada - fecha_pago).days
        
        #Creo una nueva fila con los datos faltantes
        nueva_fila = pd.DataFrame([{
            "Año_mes": mes,
            "Fecha de pago esperada": fecha_pago_esperada.strftime('%Y-%m-%d'),
            "Fecha de Pago": fecha_pago.strftime('%Y-%m-%d'),
            "Delta de pago": delta_pago
        }])

        #Agrego la nueva fila al df
        df = pd.concat([df, nueva_fila], ignore_index=True)

#Función para incrementar el mes en la columna "Año_mes" para que tenga coherencia el nombre con la fecha de pago. Ejemplo: Fecha de pago: 2020-04-29 corresponde al mes 2020_05.
def incrementar_mes(row):
    #Obtengo el año y mes de la columna "Año_mes"
    year, month = map(int, row['Año_mes'].split('_'))
    if month == 12:  #Si el mes es diciembre, paso a enero del año siguiente
        year += 1
        month = 1
    else:
        month += 1  #Si no es diciembre, solo incremento el mes
    return f"{year}_{month:02d}"

#Aplico la función que incrementa el mes a cada fila del df
df['Año_mes'] = df.apply(incrementar_mes, axis=1)

#Ordeno nuevamente el df por "Año_mes"
df = df.sort_values(by="Año_mes").reset_index(drop=True)

#Imprimo df
print(df)

#Defino ruta para exportar a un .CSV para visualizarlo en Power BI
output_csv_path = r"C:\Users\Federico Labate\Desktop\Pagos_de_alquiler\Output.csv"

#Exporto el df a un archivo CSV sin los ìndices
df.to_csv(output_csv_path, index=False)


    Año_mes Fecha de pago esperada Fecha de Pago  Delta de pago
0   2020_05             2020-05-05    2020-04-29              6
1   2020_06             2020-06-05    2020-05-30              6
2   2020_07             2020-07-05    2020-06-29              6
3   2020_08             2020-08-05    2020-07-30              6
4   2020_09             2020-09-05    2020-08-29              7
5   2020_10             2020-10-05    2020-09-30              5
6   2020_11             2020-11-05    2020-10-29              7
7   2020_12             2020-12-05    2020-11-30              5
8   2021_01             2021-01-05    2020-12-28              8
9   2021_02             2021-02-05    2021-01-29              7
10  2021_03             2021-03-05    2021-02-26              7
11  2021_04             2021-04-05    2021-03-30              6
12  2021_05             2021-05-05    2021-04-29              6
13  2021_06             2021-06-05    2021-05-31              5
14  2021_07             2021-07-05    20

OSError: Cannot save file into a non-existent directory: 'C:\Users\Federico Labate\Desktop\Pagos_de_alquiler'